In [2]:
import os
train_path = "D:/work/car-work/car/dataset/image/train"
image_count = len([f for f in os.listdir(train_path) if f.endswith(('.jpg', '.png'))])
print(f"训练集图像数量: {image_count}")

训练集图像数量: 14


In [8]:
import os

# 替换为你的实际路径
image_dir = r"D:\work\car-work\car\dataset\image\train"
label_dir = r"D:\work\car-work\car\dataset\label\train"

# 获取所有图像文件
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# 检查每个图像是否有对应的标签文件
missing_labels = []
empty_labels = []

for img_file in image_files:
    # 获取对应的标签文件名
    base_name = os.path.splitext(img_file)[0]
    label_file = f"{base_name}.txt"
    label_path = os.path.join(label_dir, label_file)
    
    # 检查标签文件是否存在
    if not os.path.exists(label_path):
        missing_labels.append(img_file)
    else:
        # 检查标签文件是否为空
        if os.path.getsize(label_path) == 0:
            empty_labels.append(img_file)

# 输出结果
if missing_labels:
    print(f"发现 {len(missing_labels)} 个图像没有对应的标签文件:")
    for f in missing_labels[:10]:  # 只打印前10个
        print(f"  - {f}")
else:
    print("所有图像都有对应的标签文件")

if empty_labels:
    print(f"发现 {len(empty_labels)} 个空的标签文件:")
    for f in empty_labels[:10]:
        print(f"  - {f}")

所有图像都有对应的标签文件


In [11]:
import os

label_dir = r"D:\work\car-work\car\dataset\label\train"
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

empty_files = 0
invalid_format = 0
invalid_coords = 0

for label_file in label_files:
    file_path = os.path.join(label_dir, label_file)
    
    # 检查文件是否为空
    if os.path.getsize(file_path) == 0:
        empty_files += 1
        continue
    
    # 检查内容格式
    with open(file_path, 'r') as f:
        lines = f.readlines()
        
        for line in lines:
            parts = line.strip().split()
            
            # 检查格式是否符合OBB要求（至少6个参数）
            if len(parts) < 6:
                invalid_format += 1
                continue
                
            # 检查坐标是否在有效范围内
            try:
                x_center = float(parts[1])
                y_center = float(parts[2])
                width = float(parts[3])
                height = float(parts[4])
                angle = float(parts[5])
                
                # 检查归一化坐标范围
                if not (0 <= x_center <= 1 and 0 <= y_center <= 1):
                    invalid_coords += 1
                    print(f"文件 {label_file} 包含无效坐标: {x_center}, {y_center}")
                    
                # 检查宽高是否为正
                if width <= 0 or height <= 0:
                    invalid_coords += 1
                    print(f"文件 {label_file} 包含无效宽高: {width}, {height}")
                    
            except ValueError:
                invalid_format += 1
                print(f"文件 {label_file} 包含非数字值")

print(f"检查结果:")
print(f"空标签文件: {empty_files}")
print(f"格式错误的标签: {invalid_format}")
print(f"坐标超出范围的标签: {invalid_coords}")

检查结果:
空标签文件: 0
格式错误的标签: 0
坐标超出范围的标签: 0


In [12]:
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO("yolo11n-obb.pt").to("cpu")
    results = model.train(
        data="D:/work/car-work/car/dataset/dataset.yaml",
        epochs=10,       # 调整为30轮
        batch=4,         # 调整为4
        imgsz=640        # 输入尺寸可保持默认，或降低至320进一步减少计算量
    )

engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/work/car-work/car/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-obb.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, reti

train: Scanning D:\work\car-work\car\dataset\image\train.cache... 0 images, 14 backgrounds, 0 corrupt: 100%|██████████| 14/14 [00:00<?, ?it/s]

WARNING Labels are missing or empty in D:\work\car-work\car\dataset\image\train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: Fast image access  (ping: 0.00.0 ms, read: 2022.0500.2 MB/s, size: 661.8 KB)



val: Scanning D:\work\car-work\car\dataset\image\val.cache... 0 images, 4 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

WARNING Labels are missing or empty in D:\work\car-work\car\dataset\image\val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs\obb\train12\labels.jpg... 
WARNING zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 87 weight(decay=0.0), 97 weight(decay=0.0005), 96 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\obb\train12
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all          4          0          0          0          0          0
WARNING no labels found in obb set, can not compute metrics without labels



c:\Users\11746\miniconda3\envs\yoloobb\lib\site-packages\ultralytics\utils\metrics.py:765: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
c:\Users\11746\miniconda3\envs\yoloobb\lib\site-packages\numpy\_core\_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/10         0G          0      65.19          0          0        640:  75%|███████▌  | 3/4 [00:03<00:01,  1.26s/it]


KeyboardInterrupt: 